In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

# SAM 3 を使用したインタラクティブなインスタンスセグメンテーション (Interactive Instance Segmentation using SAM 3)


## Google Colab セットアップ

Google Colabで実行する場合は、以下のセルを実行してください。
これにより、必要なライブラリのインストールと、データの読み込み準備が行われます。

In [ ]:
# Google Colab環境のセットアップ
try:
    import google.colab
    IN_COLAB = True
    print("Google Colab環境で実行中")
except ImportError:
    IN_COLAB = False
    print("ローカル環境で実行中")

if IN_COLAB:
    # 1. Google Driveをマウント (推奨)
    # 自分のデータやコードを使用する場合は、Google Driveにアップロードしてマウントします
    from google.colab import drive
    drive.mount('/content/drive')
    
    # 2. SAM3のインストールとパス設定
    import os
    import sys
    
    # Google Drive内のSAM3ディレクトリのパス
    # ※ご自身の環境に合わせてパスを変更してください
    DRIVE_SAM3_PATH = "/content/drive/MyDrive/sam3"
    
    if os.path.exists(DRIVE_SAM3_PATH):
        print(f"Google Drive内のSAM3が見つかりました: {DRIVE_SAM3_PATH}")
        os.chdir(DRIVE_SAM3_PATH)
        print("カレントディレクトリを変更しました。")
        
        # 依存関係のインストール
        print("依存関係をインストールしています...")
        # Numpy 2.0との互換性問題を回避するためにバージョンを固定
        !pip install -q "numpy<2.0"
        !pip install -q -e .
        !pip install -q pycocotools
        
    else:
        print(f"Google Drive内に {DRIVE_SAM3_PATH} が見つかりませんでした。")
        print("GitHubからSAM3をクローンしてインストールします...")
        
        # GitHubからクローン
        if not os.path.exists("/content/sam3"):
            !git clone https://github.com/facebookresearch/sam3.git /content/sam3
            
        os.chdir("/content/sam3")
        print("カレントディレクトリを /content/sam3 に変更しました。")
        
        # 依存関係のインストール
        # Numpy 2.0との互換性問題を回避するためにバージョンを固定
        !pip install -q "numpy<2.0"
        !pip install -q -e .
        !pip install -q pycocotools
    
    print("セットアップが完了しました。")
    print(f"現在の作業ディレクトリ: {os.getcwd()}")

### データの使用について

- **Google Driveを使用する場合**: `GT_DIR` や `PRED_DIR` には `/content/drive/MyDrive/...` から始まるパスを指定してください。
- **GitHubからクローンした場合**: 左側のファイルブラウザから `/content/sam3` 内を確認できます。データは別途アップロードが必要です。

In [ ]:
# Google Colab環境のセットアップ
try:
    import google.colab
    IN_COLAB = True
    print("Google Colab環境で実行中")
except ImportError:
    IN_COLAB = False
    print("ローカル環境で実行中")

if IN_COLAB:
    # 1. Google Driveをマウント (推奨)
    # 自分のデータやコードを使用する場合は、Google Driveにアップロードしてマウントします
    from google.colab import drive
    drive.mount('/content/drive')
    
    # 2. SAM3のインストールとパス設定
    import os
    import sys
    
    # Google Drive内のSAM3ディレクトリのパス
    # ※ご自身の環境に合わせてパスを変更してください
    DRIVE_SAM3_PATH = "/content/drive/MyDrive/sam3"
    
    if os.path.exists(DRIVE_SAM3_PATH):
        print(f"Google Drive内のSAM3が見つかりました: {DRIVE_SAM3_PATH}")
        os.chdir(DRIVE_SAM3_PATH)
        print("カレントディレクトリを変更しました。")
        
        # 依存関係のインストール
        print("依存関係をインストールしています...")
        !pip install -q -e .
        !pip install -q pycocotools
        
    else:
        print(f"Google Drive内に {DRIVE_SAM3_PATH} が見つかりませんでした。")
        print("GitHubからSAM3をクローンしてインストールします...")
        
        # GitHubからクローン
        if not os.path.exists("/content/sam3"):
            !git clone https://github.com/facebookresearch/sam3.git /content/sam3
            
        os.chdir("/content/sam3")
        print("カレントディレクトリを /content/sam3 に変更しました。")
        
        # 依存関係のインストール
        !pip install -q -e .
        !pip install -q pycocotools
    
    print("セットアップが完了しました。")
    print(f"現在の作業ディレクトリ: {os.getcwd()}")

### データの使用について

- **Google Driveを使用する場合**: `GT_DIR` や `PRED_DIR` には `/content/drive/MyDrive/...` から始まるパスを指定してください。
- **GitHubからクローンした場合**: 左側のファイルブラウザから `/content/sam3` 内を確認できます。データは別途アップロードが必要です。

Segment Anything Model 3 (SAM 3) は、幾何学的なプロンプト (SAM 1 タスク) を与えられると、目的のオブジェクトを示すインスタンスマスクを予測します。
`SAM3Image` クラスと `Sam3Processor` クラスは、モデルにプロンプトを与えるための簡単なインターフェースを提供します。
ユーザーはまず `Sam3Processor.set_image` メソッドを使用して画像を設定し、必要な画像埋め込み (embeddings) を計算します。
その後、`predict` メソッドを介してプロンプトを提供することで、それらのプロンプトからマスクを効率的に予測できます。
モデルは入力として、ポイントプロンプトとボックスプロンプトの両方、および前の予測イテレーションからのマスクを受け取ることができます。

このノートブックは、インタラクティブな画像セグメンテーションのための SAM 2 API に従っています。

# <a target="_blank" href="https://colab.research.google.com/github/facebookresearch/sam3/blob/main/notebooks/sam3_for_sam1_task_example.ipynb">
#   <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
# </a>


## 環境設定 (Environment Set-up)


まず、リポジトリの [インストール手順](https://github.com/facebookresearch/sam3?tab=readme-ov-file#installation) に従って、環境に `sam3` をインストールしてください。

## セットアップ (Set-up)


必要なインポートと、ポイント、ボックス、マスクを表示するためのヘルパー関数を定義します。

In [ ]:
using_colab = False

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib scikit-learn
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/sam3.git'

In [ ]:
import os
# Apple MPSを使用している場合、サポートされていない操作についてはCPUにフォールバックします
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
import sam3
sam3_root = os.path.join(os.path.dirname(sam3.__file__), "..")


In [ ]:
# 計算に使用するデバイスを選択
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # ノートブック全体でbfloat16を使用
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # Ampere GPU向けにtfloat32を有効化 (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 3 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )

In [ ]:
np.random.seed(3)

def show_mask(mask, ax, random_color=False, borders = True):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask = mask.astype(np.uint8)
    mask_image =  mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    if borders:
        import cv2
        contours, _ = cv2.findContours(mask,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
        # 輪郭を滑らかにする
        contours = [cv2.approxPolyDP(contour, epsilon=0.01, closed=True) for contour in contours]
        mask_image = cv2.drawContours(mask_image, contours, -1, (1, 1, 1, 0.5), thickness=2) 
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0, 0, 0, 0), lw=2))    

def show_masks(image, masks, scores, point_coords=None, box_coords=None, input_labels=None, borders=True):
    for i, (mask, score) in enumerate(zip(masks, scores)):
        plt.figure(figsize=(10, 10))
        plt.imshow(image)
        show_mask(mask, plt.gca(), borders=borders)
        if point_coords is not None:
            assert input_labels is not None
            show_points(point_coords, input_labels, plt.gca())
        if box_coords is not None:
            # ボックス
            show_box(box_coords, plt.gca())
        if len(scores) > 1:
            plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
        plt.axis('off')
        plt.show()

## 画像の例 (Example image)


In [ ]:
image = Image.open(f"{sam3_root}/assets/images/truck.jpg")

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis('on')
plt.show()

## SAM 3 でオブジェクトを選択する (Selecting objects with SAM 3)


まず、SAM 3 モデルを読み込みます。最良の結果を得るには、CUDA上で実行し、デフォルトのモデルを使用することをお勧めします。

In [ ]:
from sam3 import build_sam3_image_model
from sam3.model.sam3_image_processor import Sam3Processor

bpe_path = f"{sam3_root}/assets/bpe_simple_vocab_16e6.txt.gz"
model = build_sam3_image_model(bpe_path=bpe_path, enable_inst_interactivity=True)


`Sam3Processor.set_image` を呼び出して画像を処理し、画像埋め込み (image embedding) を生成します。

In [ ]:
processor = Sam3Processor(model)
inference_state = processor.set_image(image)

トラックを選択するには、トラック上の点を選択します。点は (x,y) 形式でモデルに入力され、ラベル 1 (前景点) または 0 (背景点) が付与されます。
複数の点を入力できますが、ここでは1つだけ使用します。選択された点は画像上に星印で表示されます。

In [ ]:
input_point = np.array([[520, 375]])
input_label = np.array([1])

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image)
show_points(input_point, input_label, plt.gca())
plt.axis('on')
plt.show()  

`SAM3Image.predict_inst` で予測を行います。モデルはマスク、それらのマスクの品質予測 (scores)、および次の予測イテレーションに渡すことができる低解像度のマスクロジット (logits) を返します。

In [ ]:
masks, scores, logits = model.predict_inst(
    inference_state,
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=True,
)
sorted_ind = np.argsort(scores)[::-1]
masks = masks[sorted_ind]
scores = scores[sorted_ind]
logits = logits[sorted_ind]

`multimask_output=True` (デフォルト設定) の場合、SAM 3 は3つのマスクを出力します。ここで `scores` は、これらのマスクの品質に対するモデル自身の推定値を与えます。
この設定は曖昧な入力プロンプトを対象としており、モデルがプロンプトと一致する異なるオブジェクトを区別するのに役立ちます。
`False` の場合、単一のマスクを返します。
単一の点のような曖昧なプロンプトの場合、単一のマスクのみが必要な場合でも `multimask_output=True` を使用することをお勧めします。
`scores` で返された最高スコアのマスクを選択することで、最適な単一マスクを選択できます。これにより、多くの場合、より良いマスクが得られます。

In [ ]:
masks.shape  # (マスクの数) x H x W

In [ ]:
show_masks(image, masks, scores, point_coords=input_point, input_labels=input_label, borders=True)

## 追加のポイントで特定のオブジェクトを指定する (Specifying a specific object with additional points)


単一の入力点は曖昧であり、モデルはそれと一致する複数のオブジェクトを返しました。単一のオブジェクトを取得するには、複数の点を提供できます。
利用可能な場合、前のイテレーションからのマスクをモデルに提供して、予測を支援することもできます。
複数のプロンプトで単一のオブジェクトを指定する場合、`multimask_output=False` を設定することで単一のマスクを要求できます。

In [ ]:
input_point = np.array([[500, 375], [1125, 625]])
input_label = np.array([1, 1])

mask_input = logits[np.argmax(scores), :, :]  # モデルの最良のマスクを選択

In [ ]:
masks, scores, _ = model.predict_inst(
    inference_state,
    point_coords=input_point,
    point_labels=input_label,
    mask_input=mask_input[None, :, :],
    multimask_output=False,
)

In [ ]:
masks.shape

In [ ]:
show_masks(image, masks, scores, point_coords=input_point, input_labels=input_label)

車を除外して窓だけを指定するには、背景点 (ラベル 0、ここでは赤で表示) を指定できます。

In [ ]:
input_point = np.array([[500, 375], [1125, 625]])
input_label = np.array([1, 0])

mask_input = logits[np.argmax(scores), :, :]  # モデルの最良のマスクを選択

In [ ]:
masks, scores, _ = model.predict_inst(
    inference_state,
    point_coords=input_point,
    point_labels=input_label,
    mask_input=mask_input[None, :, :],
    multimask_output=False,
)

In [ ]:
show_masks(image, masks, scores, point_coords=input_point, input_labels=input_label)

## ボックスで特定のオブジェクトを指定する (Specifying a specific object with a box)


モデルは、xyxy形式で提供されるボックスを入力として受け取ることもできます。

In [ ]:
input_box = np.array([425, 600, 700, 875])

In [ ]:
masks, scores, _ = model.predict_inst(
    inference_state,
    point_coords=None,
    point_labels=None,
    box=input_box[None, :],
    multimask_output=False,
)

In [ ]:
show_masks(image, masks, scores, box_coords=input_box)

## ポイントとボックスの組み合わせ (Combining points and boxes)


ポイントとボックスは、予測器に両方のタイプのプロンプトを含めるだけで組み合わせることができます。
ここでは、ホイール全体ではなく、トラックのタイヤだけを選択するために使用できます。

In [ ]:
input_box = np.array([425, 600, 700, 875])
input_point = np.array([[575, 750]])
input_label = np.array([0])

In [ ]:
masks, scores, logits = model.predict_inst(
    inference_state,
    point_coords=input_point,
    point_labels=input_label,
    box=input_box,
    multimask_output=False,
)

In [ ]:
show_masks(image, masks, scores, box_coords=input_box, point_coords=input_point, input_labels=input_label)

## バッチプロンプト入力 (Batched prompt inputs)


`SAM3Image` は、`predict_inst` メソッドを使用して、同じ画像に対して複数の入力プロンプトを受け取ることができます。
たとえば、物体検出器からの複数のボックス出力がある場合を想像してください。

In [ ]:
input_boxes = np.array([
    [75, 275, 1725, 850],
    [425, 600, 700, 875],
    [1375, 550, 1650, 800],
    [1240, 675, 1400, 750],
])

In [ ]:
masks, scores, _ = model.predict_inst(
    inference_state,
    point_coords=None,
    point_labels=None,
    box=input_boxes,
    multimask_output=False,
)

In [ ]:
masks.shape  # (バッチサイズ) x (入力ごとの予測マスク数) x H x W

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image)
for mask in masks:
    show_mask(mask.squeeze(0), plt.gca(), random_color=True)
for box in input_boxes:
    show_box(box, plt.gca())
plt.axis('off')
plt.show()

## エンドツーエンドのバッチ推論 (End-to-end batched inference)

すべてのプロンプトが事前に利用可能な場合は、SAM 3 をエンドツーエンドで直接実行することが可能です。これにより、画像に対するバッチ処理も可能になります。

In [ ]:
image1 = image  # 上記の truck.jpg
image1_boxes = np.array([
    [75, 275, 1725, 850],
    [425, 600, 700, 875],
    [1375, 550, 1650, 800],
    [1240, 675, 1400, 750],
])

image2 = Image.open(f"{sam3_root}/assets/images/groceries.jpg")
image2_boxes = np.array([
    [450, 170, 520, 350],
    [350, 190, 450, 350],
    [500, 170, 580, 350],
    [580, 170, 640, 350],
])

img_batch = [image1, image2]
boxes_batch = [image1_boxes, image2_boxes]

In [ ]:
inference_state = processor.set_image_batch(img_batch)

In [ ]:
masks_batch, scores_batch, _ = model.predict_inst_batch(
    inference_state,
    None,
    None, 
    box_batch=boxes_batch, 
    multimask_output=False
)

In [ ]:
for image, boxes, masks in zip(img_batch, boxes_batch, masks_batch):
    plt.figure(figsize=(10, 10))
    plt.imshow(image)   
    for mask in masks:
        show_mask(mask.squeeze(0), plt.gca(), random_color=True)
    for box in boxes:
        show_box(box, plt.gca())

同様に、画像のバッチに対して定義されたポイントプロンプトのバッチを持つこともできます。

In [ ]:
image1 = image  # 上記の truck.jpg
image1_pts = np.array([
    [[500, 375]],
    [[650, 750]]
    ]) # Bx1x2 ここでBはオブジェクトの数に対応します
image1_labels = np.array([[1], [1]])

image2_pts = np.array([
    [[400, 300]],
    [[630, 300]],
])
image2_labels = np.array([[1], [1]])

pts_batch = [image1_pts, image2_pts]
labels_batch = [image1_labels, image2_labels]

In [ ]:
masks_batch, scores_batch, _ = model.predict_inst_batch(inference_state, pts_batch, labels_batch, box_batch=None, multimask_output=True)

# オブジェクトごとに最適な単一マスクを選択
best_masks = []
for masks, scores in zip(masks_batch,scores_batch):
    best_masks.append(masks[range(len(masks)), np.argmax(scores, axis=-1)])

In [ ]:
for image, points, labels, masks in zip(img_batch, pts_batch, labels_batch, best_masks):
    plt.figure(figsize=(10, 10))
    plt.imshow(image)   
    for mask in masks:
        show_mask(mask, plt.gca(), random_color=True)
    show_points(points, labels, plt.gca())